In [21]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import sklearn
import swifter
import numpy as np
import lightgbm
from sklearn.model_selection import train_test_split
import joblib
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.linear_model import LogisticRegression, Ridge, RidgeClassifier

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [22]:
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [23]:
rheed_long = pd.read_csv("../data/onti_p.csv")

In [24]:
#проверка на корректную склейку
print(rheed_long.shape)
rheed_long[:1]

(2363772, 18)


,Unnamed: 0,Unnamed: 0.1,Coolness_RHEED,Length_RHEED,R FWHM_RHEED,Speed_RHEED,Time_RHEED,X FWHM_RHEED,Y FWHM_RHEED,Crystal Position,Displayed Rate,Filtered Rate,Fundamental Freq,Length,Raw Rate,Source Power,Speed,Time
0,0,0.0,0.0,0.003721,0.0,0.040601,2021-02-23 02:21:05,0.0,0.0,1.0,6.025618,5.999182,5.967367e+06,0.003721,5.987163,21.635099,NaN,2021-02-23 02:21:05


In [25]:
#rheed_long = df_comma_to_float(rheed_long)
rheed_long.shape

(2363772, 18)

In [26]:
#важные колонки для использования
val_col = ['Coolness_RHEED',  'R FWHM_RHEED',  'X FWHM_RHEED', 'Y FWHM_RHEED',
        'Filtered Rate', 'Displayed Rate', 'Raw Rate',
       'Source Power', 'Crystal Position']


In [27]:
#создаем датасет разными аггрегационными функциями
def generated_dataset(series, wind_size=5):
    result = series.rolling(wind_size).agg(["mean", "std", "var"])
    print(result.shape) 
    delta = series - series.shift(wind_size)
    print(delta.shape)
    result = result.join(delta)
    result.rename(columns={c:str(c)+"_"+str(wind_size) for c in result.columns}, inplace=True)

    print(result.shape)

    return result

In [28]:
df_X = generated_dataset(rheed_long[val_col])
df_X.shape, df_X.columns

(2363772, 27)
(2363772, 9)


/home/ec2-user/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


(2363772, 36)


((2363772, 36),
 Index(['('Coolness_RHEED', 'mean')_5', '('Coolness_RHEED', 'std')_5',
        '('Coolness_RHEED', 'var')_5', '('R FWHM_RHEED', 'mean')_5',
        '('R FWHM_RHEED', 'std')_5', '('R FWHM_RHEED', 'var')_5',
        '('X FWHM_RHEED', 'mean')_5', '('X FWHM_RHEED', 'std')_5',
        '('X FWHM_RHEED', 'var')_5', '('Y FWHM_RHEED', 'mean')_5',
        '('Y FWHM_RHEED', 'std')_5', '('Y FWHM_RHEED', 'var')_5',
        '('Filtered Rate', 'mean')_5', '('Filtered Rate', 'std')_5',
        '('Filtered Rate', 'var')_5', '('Displayed Rate', 'mean')_5',
        '('Displayed Rate', 'std')_5', '('Displayed Rate', 'var')_5',
        '('Raw Rate', 'mean')_5', '('Raw Rate', 'std')_5',
        '('Raw Rate', 'var')_5', '('Source Power', 'mean')_5',
        '('Source Power', 'std')_5', '('Source Power', 'var')_5',
        '('Crystal Position', 'mean')_5', '('Crystal Position', 'std')_5',
        '('Crystal Position', 'var')_5', 'Coolness_RHEED_5', 'R FWHM_RHEED_5',
        'X FWHM_RHEED_5', '

In [29]:
df_X = df_X.join(generated_dataset(rheed_long[val_col],wind_size=50))#,rsuffix="_100_")
df_X.shape

(2363772, 27)
(2363772, 9)
(2363772, 36)


(2363772, 72)

In [30]:
df_X = df_X.join(generated_dataset(rheed_long[val_col],wind_size=500))#,rsuffix="_500_")
df_X.shape

(2363772, 27)
(2363772, 9)
(2363772, 36)


(2363772, 108)

In [31]:
df_X = df_X.join(generated_dataset(rheed_long[val_col],wind_size=1500))#,rsuffix="_1500_")
df_X.shape

(2363772, 27)
(2363772, 9)
(2363772, 36)


(2363772, 144)

In [32]:
mask_base = df_X.isna().sum(axis=1)==0
#df_X.fill(inplace=True)
df_X.shape

(2363772, 144)

In [33]:
import random
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
wind_size = 435

df = df_X[wind_size:-wind_size]

cool_col = '(\'Coolness_RHEED\', \'mean\')_50'
y2 = pd.Series((abs(rheed_long['Coolness_RHEED'] - df[cool_col].shift(-wind_size))>5)[435:2363337])
y = ((df[cool_col] - df[cool_col].shift(-wind_size))>5)
y.index, y2.index

(RangeIndex(start=435, stop=2363337, step=1),
 Int64Index([    435,     436,     437,     438,     439,     440,     441,
                 442,     443,     444,
             ...
             2363327, 2363328, 2363329, 2363330, 2363331, 2363332, 2363333,
             2363334, 2363335, 2363336],
            dtype='int64', length=2362902))

In [34]:
index_list = random.sample([ i for i in y[(y==1)].index.tolist() if i > 10001],18000)

index_list+=random.sample([ i for i in y[(y==0)].index.tolist() if i > 10001],25000)
df = df.loc[index_list]
y = y.loc[index_list]
mask = mask_base #&(y>30)
df = df[mask]
y=y[mask]
categorical_features_indices = np.where(df.dtypes != np.float)[0]
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.0005, shuffle=False)
#X_train = df
#y_train = y

/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


In [35]:
import hyperopt

def hyperopt_objective(params):
    model = CatBoostClassifier(
#        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=13,
        eval_metric='AUC',
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
    )
    
    cv_data = cv(
        Pool(X_train, y_train, cat_features=categorical_features_indices),
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-AUC-mean'])
    
    return 1 - best_accuracy # as hyperopt minimises

In [ ]:
from numpy.random import RandomState

params_space = {
#    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-4, 1e-1)
#     'colsample_bylevel': hyperopt.hp.uniform('colsample_bylevel', 0.5, 1.0),
#     'bagging_temperature': hyperopt.hp.uniform('bagging_temperature', 0.0, 100),
#     'random_strength': hyperopt.hp.uniform('random_strength', 0.0, 100),
#     'n_estimators': hyperopt.hp.choice('n_estimators', np.arange(50, 250, 25)),
#     'max_depth': hyperopt.hp.choice('max_depth', np.arange(5, 11)),
#     'reg_lambda': hyperopt.hp.uniform('reg_lambda', 1, 10),
#     'scale_pos_weight': hyperopt.hp.uniform('scale_pos_weight', 1, 50)
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=500,
    trials=trials,
    rstate=RandomState(123)
)

print(best)

 25%|██▌       | 125/500 [04:01<15:25,  2.47s/trial, best loss: 0.07854494621933972]

In [ ]:
model = CatBoostClassifier(
    #l2_leaf_reg=int(best['l2_leaf_reg']),
    learning_rate=best['learning_rate'],
    iterations=250,
    eval_metric='AUC',
    random_seed=42,
    verbose=True,
    loss_function='Logloss',
    #od_type='Iter',
    #od_wait=250
)
# Learning rate 0.035394

# cv_params = model.get_params()
# cv_data = cv(
#     Pool(df, y, cat_features=categorical_features_indices),
#     cv_params,
#     plot=True,
#     seed=42
# )

# cv_data

In [ ]:
# print('Best validation AUC score: {:.2f}±{:.2f} on step {}'.format(
#     np.max(cv_data['test-AUC-mean']),
#     cv_data['test-AUC-std'][np.argmax(cv_data['test-AUC-mean'])],
#     np.argmax(cv_data['test-AUC-mean'])
# ))

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
#validate_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

model.fit(train_pool)#, eval_set=validate_pool)

In [ ]:
fpr, tpr, _ = sklearn.metrics.roc_curve(y_test, model.predict_proba(X_test[model_cols])[:,1])
roc_auc = sklearn.metrics.roc_auc_score(y_test, model.predict_proba(X_test[model_cols])[:,1])
plt.figure()
lw = 2
plt.plot(fpr, tpr,color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
roc_auc

In [ ]:
import pickle
with open("../saves/catboost/catboostn_1.pkl","wb") as f:
    pickle.dump(model, f)